In [ ]:
!pip install py_pdf_parser

In [ ]:
from py_pdf_parser.loaders import load_file
import requests
import csv
import pandas as pd

In [ ]:
pdf_version_from_url = requests.get('https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000386170')

with open('inmate.pdf', 'wb') as pdf:
  pdf.write(pdf_version_from_url.content)

In [ ]:
document = load_file('inmate.pdf')

In [ ]:
print([element.text() for element in document.elements])

['INMATE SEARCH DETAIL REPORT', 'AARON, SAMUEL EARNEST (00387497)', 'INMATE DESCRIPTION', 'INMATE SENTENCE AND LOCATION', 'SEX:', 'MALE', 'SCDC ID:', '00387497', 'RACE:', 'WHITE', 'SID:', 'SC02390399', 'HEIGHT:', '6\' 03"', 'OFFENDER TYPE:', 'ADULT-STRAIGHT SENTENCE', 'WEIGHT:', '192 lbs.', 'ADMISSION DATE:', '03/23/2022', 'AGE:', '55', 'LOCATION:', 'RIDGELAND', 'CITIZENSHIP:', 'CITIZEN - NATIVE BORN', 'DORM-ROOM-BUNK:', 'BA-0007-B', 'BUILD:', 'LARGE', 'EWC LEVEL:', '2F5', 'COMPLEXION:', 'FAIR', 'EEC LEVEL:', 'HAIR COLOR:', 'GRAY OR PARTIALLY GRAY', 'PROJECTED RELEASE DATE:', '01/31/2037', 'EYE COLOR:', 'HAZEL', 'PROJECTED PAROLE ELIGIBILITY:', 'NOT ELIGIBLE', 'PICTURE DATE:', '03/23/2022', 'SUP. FURLOUGH ELIGIBILITY:', 'NOT ELIGIBLE', 'CURRENT OFFENSES', 'OFFENSE', 'SENTENCE TYPE', 'YRS', 'MOS', 'DYS', 'COUNTY', 'START DATE', 'V/NV', 'CAT', 'INDICT', 'STATUS', 'STRONG ARM ROBBERY', 'ADULT-STRAIGHT SENTENCE', '15', '0', '0', 'LEXINGTON', '02/05/2020', 'N', '3', '20GS3201839', 'ACTIVE',

In [ ]:
name_element = document.elements.filter_by_text_equal('INMATE SEARCH DETAIL REPORT')[0]
name_text = document.elements.to_the_right_of(name_element).extract_single_element().text()
print(name_text)

AARON, SAMUEL EARNEST (00387497)


In [ ]:
sid_element = document.elements.filter_by_text_equal('SID:')[0]
sid_text = document.elements.to_the_right_of(sid_element)[0].text()
print(sid_text)

SC02427358


In [ ]:
#if there's no escapes and it reaches last element on page, remove the last element from list and continue with second page

number_of_pages_for_offenses = len(document.elements.filter_by_text_equal('OFFENSE'))

for n in range(0, number_of_pages_for_offenses):
  element = document.elements.filter_by_text_equal('OFFENSE')[n]
  i = 0
  offense = text = document.elements.below(element)[i]
  while (text.text() != 'ESCAPES'):
    i += 1
    try:
      text = document.elements.below(element)[i]
      print(text.text())
    except:
      #remove last i term added
      print(text.text())
      break


In [ ]:
# MOVEMENT
number_of_pages_for_movement = len(document.elements.filter_by_text_equal('MOVEMENT DATE'))

for n in range(0, number_of_pages_for_movement):
  element = document.elements.filter_by_text_equal('MOVEMENT DATE')[n]
  i = 0
  movement = text = document.elements.below(element)[i]
  while (text.text() != 'EARNED WORK CREDITS'):
    i += 1
    try:
      text = document.elements.below(element)[i]
      print(text.text())
    except:
      #remove last i term added
      print('REMOVE', text.text())
      break


06/28/2023
06/12/2023
06/12/2023
03/27/2023
03/27/2023
July 24, 2023
REMOVE July 24, 2023
03/07/2023
12/09/2021
12/03/2021
12/03/2021
12/02/2021
12/02/2021
12/02/2021
09/30/2021
EARNED WORK CREDITS


In [ ]:
print(len(document.elements.filter_by_text_equal('OFFENSE')))

2


In [ ]:
print(sex_text[2].text())

INSTIT TRANSFER


In [ ]:
features_to_retrieve = ['SID:', 'SEX:', 'RACE:', 'HEIGHT:', 'WEIGHT:', 'AGE:', 'COMPLEXION:', 'HAIR COLOR:',
                        'EYE COLOR:', 'ADMISSION DATE:', 'LOCATION:', #current prison location
                        'PROJECTED RELEASE DATE:', 'PROJECTED PAROLE ELIGIBILITY:']

for feature in features_to_retrieve:
  element = document.elements.filter_by_text_equal(feature)[0]
  text = document.elements.to_the_right_of(element)[0].text()
  print(text)



SC02390399
MALE
WHITE
6' 03"
192 lbs.
55
FAIR
GRAY OR PARTIALLY GRAY
HAZEL
03/23/2022
RIDGELAND
01/31/2037
NOT ELIGIBLE


In [ ]:
#Continue fetching offense till it reaches ESCAPES
element = document.elements.filter_by_text_equal('OFFENSE')[0]
i = 0
offense = text = document.elements.below(element)[i]

# while (text != 'ESCAPES'):
#   i += 1
#   text = document.elements.below(element)[i].text()
#   if (text != 'ESCAPES'):
#     offense = offense + ', ' + text

# print(offense)

#Creating a different table for offenses
offenses_fields = ['SID', 'OFFENSE', 'SENTENCE TYPE', 'YEARS', 'MONTHS', 'DAYS', 'COUNTY', 'START DATE', 'STATUS']
with open('sc_inmate_offenses.csv', 'w') as offenses_file:
  o_writer = csv.writer(offenses_file)
  o_writer.writerow(offenses_fields)
  while (text.text() != 'ESCAPES'):
    offense_row = []
    offense_details = document.elements.to_the_right_of(text)
    # in offense details: 0-sentence_type, 1-yrs, 2-mnths, 3-days, 4-county, 5-start_date, 6-v/nv, 7-cat, 8-indict, 9-status
    o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), offense_details[2].text(), offense_details[3].text(),
                       offense_details[4].text(), offense_details[5].text(), offense_details[9].text()])
    i += 1
    text = document.elements.below(element)[i]


In [ ]:
element = document.elements.filter_by_text_equal('COUNTY')[0] #county of commitment
text = document.elements.below(element)[0].text() #will only get the county for first offense
print(text)

LEXINGTON


In [ ]:
images_csv = pd.read_csv('images.csv')
print(images.columns)

Index(['ID', 'Image URL'], dtype='object')


## Merged code for generating PDF links, parsing PDF, and storing in CSV



In [ ]:
images_csv = pd.read_csv('images.csv')

In [ ]:
features_to_retrieve = ['SID:', 'SEX:', 'RACE:', 'HEIGHT:', 'WEIGHT:', 'AGE:', 'COMPLEXION:', 'HAIR COLOR:',
                        'EYE COLOR:', 'ADMISSION DATE:', 'LOCATION:', #current prison location
                        'PROJECTED RELEASE DATE:', 'PROJECTED PAROLE ELIGIBILITY:']


fields = ['NAME', 'SID', 'SEX', 'RACE', 'HEIGHT', 'WEIGHT', 'AGE', 'COMPLEXION', 'HAIR COLOR',
                        'EYE COLOR', 'ADMISSION DATE', 'CURRENT LOCATION', #current prison location
                        'PROJECTED RELEASE DATE', 'PROJECTED PAROLE ELIGIBILITY']
with open('sc_inmate_details.csv', 'w') as csv_file:
  csv_writer = csv.writer(csv_file)
  csv_writer.writerow(fields)

offenses_fields = ['SID', 'OFFENSE', 'SENTENCE TYPE', 'YEARS', 'MONTHS', 'DAYS', 'COUNTY', 'START DATE', 'STATUS']
with open('sc_inmate_offenses.csv', 'w') as offenses_file:
  o_writer = csv.writer(offenses_file)
  o_writer.writerow(offenses_fields)

In [ ]:
import os
import requests
from pdfminer.pdfdocument import PDFSyntaxError

base_url = "https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%"

inmate_list = []

with open('sc_inmate_details.csv', 'a') as details_file, open('sc_inmate_offenses.csv', 'a') as offenses_file:
  csv_writer = csv.writer(details_file)
  o_writer = csv.writer(offenses_file)
  #looping through all combinations of possible IDs

  for pdf_id in images_csv['ID']:
  #for pdf_id in range(387496, 387510):
      pdf_id_str = str(pdf_id).zfill(9)  # Pad with leading zeros to make it 9 digits

      pdf_url = base_url + '2' + pdf_id_str

      try:
          # print(pdf_url)
          response = requests.get(pdf_url)
          #initiating list for each inmate to add row to csv file
          inmate_row = []
          with open('temp.pdf', 'wb') as pdf:
              pdf.write(response.content)
          try:
              document = load_file('temp.pdf')
              # code to scrap the PDF and save that in csv
              # parsing NAME first
              name_element = document.elements.filter_by_text_equal('INMATE SEARCH DETAIL REPORT')[0]
              name_text = document.elements.to_the_right_of(name_element).extract_single_element().text()
              inmate_row.append(name_text)

              # parsing rest of the features
              for feature in features_to_retrieve:
                element = document.elements.filter_by_text_equal(feature)[0]
                try:
                  text = document.elements.to_the_right_of(element)[0].text()
                except:
                  text = 'Data unavailable'
                inmate_row.append(text)

              #storing sid in a separate variable to save it in the offenses csv file
              element = document.elements.filter_by_text_equal('SID:')[0]
              sid_text = document.elements.to_the_right_of(element)[0].text()


              # STORING OFFENSES in a different csv file
              try:
                element = document.elements.filter_by_text_equal('OFFENSE')[0]

              except:
                # store no current offenses
                o_writer.writerow([sid_text, 'NO CURRENT OFFENSES/RECORD UNAVAILABLE', '', '', '', '',
                                 '', '', ''])
                continue
              i = 0
              offense = text = document.elements.below(element)[i]
              offense_details = document.elements.to_the_right_of(text)
              while (text.text() != 'ESCAPES'):
                if (len(offense_details) == 9):
                  print(pdf_url)
                  try:
                    county_date = offense_details[4].text().split(' ')

                    if (len(county_date) > 1):
                      print(county_date)
                      o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), offense_details[2].text(), offense_details[3].text(),
                                  county_date[0], county_date[1], offense_details[8].text()])
                    else:
                      # there is a trailing blank space, and there is no indict number present although it splits
                      o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), offense_details[2].text(), offense_details[3].text(),
                                 offense_details[4].text(), offense_details[5].text(), offense_details[8].text()])

                  except:
                    # if it does not split, there is no indict number present when the length is 9
                    # implies that the status is on index 8
                    o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), offense_details[2].text(), offense_details[3].text(),
                                 offense_details[4].text(), offense_details[5].text(), offense_details[8].text()])
                    print('')

                elif (len(offense_details) == 8):
                  # 8 elements implies total term is life sentence
                  o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), '', '',
                                  offense_details[4].text(), offense_details[5].text(), offense_details[7].text()])
                  print('')
                  #continue
                else:
                  #all 10 fields present
                  o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), offense_details[2].text(), offense_details[3].text(),
                                  offense_details[4].text(), offense_details[5].text(), offense_details[9].text()])

                i += 1
                text = document.elements.below(element)[i]


              #print(inmate_row)
              #adding row to list
              #inmate_list.append(inmate_row)

              #writing to csv
              csv_writer.writerow(inmate_row)

          except PDFSyntaxError:
              #print(f"PDF does not exist: {pdf_url}")
              continue

          except:
            print(pdf_url)
            continue

      except requests.exceptions.RequestException:
          pass

# adding all rows from list to csv
# with open('inmates.csv', 'a') as csv_file:
#   csv_writer = csv.writer(csv_file)
#   csv_writer.writerows(inmate_list)

https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000386170
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000250432
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000340614
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000389937
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000382830
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000268619
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000200592
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000289550
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000387463
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000390024
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000145147
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000374387
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000387538
https://public.doc.state.

In [ ]:
# for the records where offenses spans 2 pages
import os
import requests
from pdfminer.pdfdocument import PDFSyntaxError

base_url = "https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%"

inmate_list = ['386170', '250432', '340614', '389937', '382830', '268619', '200592', '289550', '387463', '390024',
               '145147', '374387', '387538', '387364', '131898', '155043', '297589', '376744', '354299', '385358',
               '386142', '253928', '234880', '385426', '384544', '220563', '347235', '223347', '283647', '242126',
               '284412']

offenses_df = pd.read_csv('sc_inmate_offenses.csv')
with open('sc_inmate_details.csv', 'a') as details_file, open('sc_inmate_offenses.csv', 'a') as offenses_file:
  csv_writer = csv.writer(details_file)
  o_writer = csv.writer(offenses_file)
  #looping through all combinations of possible IDs

  #for pdf_id in images_csv['ID']:
  #for pdf_id in range(387496, 387510):
  for pdf_id in inmate_list:
      pdf_id_str = str(pdf_id).zfill(9)  # Pad with leading zeros to make it 9 digits

      pdf_url = base_url + '2' + pdf_id_str

      try:
          # print(pdf_url)
          response = requests.get(pdf_url)
          #initiating list for each inmate to add row to csv file
          inmate_row = []
          with open('temp.pdf', 'wb') as pdf:
              pdf.write(response.content)
          try:
              document = load_file('temp.pdf')
              # code to scrap the PDF and save that in csv
              # parsing NAME first
              name_element = document.elements.filter_by_text_equal('INMATE SEARCH DETAIL REPORT')[0]
              name_text = document.elements.to_the_right_of(name_element).extract_single_element().text()
              inmate_row.append(name_text)

              # parsing rest of the features
              for feature in features_to_retrieve:
                element = document.elements.filter_by_text_equal(feature)[0]
                try:
                  text = document.elements.to_the_right_of(element)[0].text()
                except:
                  text = 'Data unavailable'
                inmate_row.append(text)

              #storing sid in a separate variable to save it in the offenses csv file
              element = document.elements.filter_by_text_equal('SID:')[0]
              sid_text = document.elements.to_the_right_of(element)[0].text()


              # STORING OFFENSES in a different csv file
              try:
                element = document.elements.filter_by_text_equal('OFFENSE')[0]
              except:
                # store no current offenses
                o_writer.writerow([sid_text, 'NO CURRENT OFFENSES/RECORD UNAVAILABLE', '', '', '', '',
                                 '', '', ''])
                continue

              number_of_pages_for_offenses = len(document.elements.filter_by_text_equal('OFFENSE'))

              for n in range(0, number_of_pages_for_offenses):
                element = document.elements.filter_by_text_equal('OFFENSE')[n]
                i = 0
                offense = text = document.elements.below(element)[i]
                offense_details = document.elements.to_the_right_of(text)
                while (text.text() != 'ESCAPES'):
                  if (len(offense_details) == 9):
                    try:
                      county_date = offense_details[4].text().split(' ')

                      if (len(county_date) > 1):
                        o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), offense_details[2].text(), offense_details[3].text(),
                                    county_date[0], county_date[1], offense_details[8].text()])
                      else:
                        # there is a trailing blank space, and there is no indict number present although it splits
                        o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), offense_details[2].text(), offense_details[3].text(),
                                  offense_details[4].text(), offense_details[5].text(), offense_details[8].text()])

                    except:
                      # if it does not split, there is no indict number present when the length is 9
                      # implies that the status is on index 8
                      o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), offense_details[2].text(), offense_details[3].text(),
                                  offense_details[4].text(), offense_details[5].text(), offense_details[8].text()])
                      print('')

                  elif (len(offense_details) == 8):
                    # 8 elements implies total term is life sentence
                    o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), '', '',
                                    offense_details[4].text(), offense_details[5].text(), offense_details[7].text()])
                    print('')
                    #continue
                  else:
                    #all 10 fields present
                    o_writer.writerow([sid_text, text.text(), offense_details[0].text(), offense_details[1].text(), offense_details[2].text(), offense_details[3].text(),
                                    offense_details[4].text(), offense_details[5].text(), offense_details[9].text()])

                  i += 1
                  try:
                    text = document.elements.below(element)[i]
                  except:
                    offenses_df.drop(offenses_df.index[-1])
                    break


              #print(inmate_row)
              #adding row to list
              #inmate_list.append(inmate_row)

              #writing to csv
              csv_writer.writerow(inmate_row)

          except PDFSyntaxError:
              #print(f"PDF does not exist: {pdf_url}")
              continue

          except:
            print(pdf_url)
          #   continue

      except requests.exceptions.RequestException:
          pass

# adding all rows from list to csv
# with open('inmates.csv', 'a') as csv_file:
#   csv_writer = csv.writer(csv_file)
#   csv_writer.writerows(inmate_list)


















https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000385426
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000384544


In [ ]:
movements_fields = ['SID', 'MOVEMENT DATE', 'TO LOCATION', 'STATUS', 'REASON']
with open('sc_inmate_movements.csv', 'w') as movements_file:
  csv_writer = csv.writer(movements_file)
  csv_writer.writerow(movements_fields)

In [ ]:
# MOVEMENT

import os
import requests
from pdfminer.pdfdocument import PDFSyntaxError

base_url = "https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%"

movements_df = pd.read_csv('sc_inmate_movements.csv')
with open('sc_inmate_movements.csv', 'a') as movements_file:
  csv_writer = csv.writer(movements_file)
  for pdf_id in images_csv['ID']:
  #for pdf_id in range(387496, 387510):
      pdf_id_str = str(pdf_id).zfill(9)  # Pad with leading zeros to make it 9 digits

      pdf_url = base_url + '2' + pdf_id_str

      #try:
          # print(pdf_url)
      response = requests.get(pdf_url)
      #initiating list for each inmate to add row to csv file
      inmate_row = []
      with open('temp.pdf', 'wb') as pdf:
          pdf.write(response.content)
          try:
            document = load_file('temp.pdf')
            element = document.elements.filter_by_text_equal('SID:')[0]
            try:
              sid_text = document.elements.to_the_right_of(element)[0].text()
            except IndexError:
              print("No SID:", pdf_url)

            try:
              number_of_pages_for_movement = len(document.elements.filter_by_text_equal('MOVEMENT DATE'))
            except:
              continue

            for n in range(0, number_of_pages_for_movement):
              element = document.elements.filter_by_text_equal('MOVEMENT DATE')[n]
              i = 0
              movement = text = document.elements.below(element)[i]
              movements_details = document.elements.to_the_right_of(text)
              while (text.text() != 'EARNED WORK CREDITS'):
                if (len(document.elements.below(element)) > i+1):
                  # print([sid_text, text.text(), movements_details[0].text(), movements_details[1].text(),
                  #                   movements_details[2].text()])
                  csv_writer.writerow([sid_text, text.text(), movements_details[0].text(), movements_details[1].text(),
                                    movements_details[2].text()])
                else:
                  # print(text.text())
                  break
                i += 1
                try:
                  text = document.elements.below(element)[i]
                  #print(text.text())
                except:
                  #remove last i term added
                  movements_df = pd.read_csv('sc_inmate_movements.csv')
                  print(movements_df.head())
                  movements_df.drop(movements_df.index[-1])
                  break

          except PDFSyntaxError:
          #print(f"PDF does not exist: {pdf_url}")
            continue

      # except:
      #   print(pdf_url)
          #   continue



No SID: https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000385426
No SID: https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000384544


In [ ]:
wc_fields = ['SID', 'JOB DESCRIPTION', 'START DATE', 'END DATE', 'TERMINATION REASON', 'JOB LEVEL']
with open('sc_inmate_work_credits.csv', 'w') as wc_file:
  csv_writer = csv.writer(wc_file)
  csv_writer.writerow(wc_fields)

In [ ]:
# EARNED WORK CREDITS

import os
import requests
from pdfminer.pdfdocument import PDFSyntaxError

base_url = "https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%"

wc_df = pd.read_csv('sc_inmate_work_credits.csv')
with open('sc_inmate_work_credits.csv', 'a') as wc_file:
  csv_writer = csv.writer(wc_file)
  for pdf_id in images_csv['ID']:
  #for pdf_id in range(387496, 387510):
      pdf_id_str = str(pdf_id).zfill(9)  # Pad with leading zeros to make it 9 digits

      pdf_url = base_url + '2' + pdf_id_str

      #try:
      print(pdf_url)
      response = requests.get(pdf_url)
      #initiating list for each inmate to add row to csv file
      inmate_row = []
      with open('temp.pdf', 'wb') as pdf:
          pdf.write(response.content)
          try:
            document = load_file('temp.pdf')
            element = document.elements.filter_by_text_equal('SID:')[0]
            try:
              sid_text = document.elements.to_the_right_of(element)[0].text()
            except IndexError:
              print("No SID:", pdf_url)

            try:
              number_of_pages_for_wc = len(document.elements.filter_by_text_equal('JOB DESCRIPTION'))
            except:
              continue

            for n in range(0, number_of_pages_for_wc):
              element = document.elements.filter_by_text_equal('JOB DESCRIPTION')[n]
              i = 0
              wc = text = document.elements.below(element)[i]
              wc_details = document.elements.to_the_right_of(text)
              while (text.text() != 'EARNED WORK CREDITS'):
                if (len(document.elements.below(element)) > i+1):
                  if len(wc_details) == 2:
                    #currently employed which is why no end date or termination reason
                    csv_writer.writerow([sid_text, text.text(), wc_details[0].text(), '', '', wc_details[1].text()])
                  # print([sid_text, text.text(), wc_details[0].text(), wc_details[1].text(),
                  #                   wc_details[2].text()])
                  else:
                    csv_writer.writerow([sid_text, text.text(), wc_details[0].text(), wc_details[1].text(),
                                    wc_details[2].text(), wc_details[3].text()])
                else:
                  # print(text.text())
                  break
                i += 1
                try:
                  text = document.elements.below(element)[i]
                  #print(text.text())
                except:
                  #remove last i term added
                  movements_df = pd.read_csv('sc_inmate_work_credits.csv')
                  print(movements_df.head())
                  movements_df.drop(movements_df.index[-1])
                  break

          except PDFSyntaxError:
          #print(f"PDF does not exist: {pdf_url}")
            continue

      # except:
      #   print(pdf_url)
          #   continue



https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000387497
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000366752
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000313799
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000248817
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000266025
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000322255
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000226996
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000389647
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000391148
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000231794
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000375427
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000388345
https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%2000346664
https://public.doc.state.

## Disciplinary Sanctions

In [ ]:
import os
import requests
from pdfminer.pdfdocument import PDFSyntaxError
from py_pdf_parser.loaders import load_file
import re
import pandas as pd
import csv
from tqdm import tqdm

headings = ['SID', 'DATE', 'DESCRIPTION', 'DETAILS']
with open('disciplinary_sanctions.csv', 'w') as discip:
    csv_writer = csv.writer(discip)
    csv_writer.writerow(headings)

images_csv = pd.read_csv('imagescopy.csv')

base_url = "https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%"

with open('disciplinary_sanctions.csv', 'a') as discip_file:
    csv_writer = csv.writer(discip_file)

    # traversing through all the IDs in the PDF
    for pdf_id in tqdm(images_csv['ID']):
        pdf_id_str = str(pdf_id).zfill(9)  # Pad with leading zeros to make it 9 digits
        pdf_url = base_url + '2' + pdf_id_str
        response = requests.get(pdf_url)
        #initiating list for each inmate to add row to csv file
        inmate_row = []
        with open('temp.pdf', 'wb') as pdf:
            pdf.write(response.content)
            try:
                document = load_file('temp.pdf')
                # element = document.elements.filter_by_text_equal('SID:')[0]

                data = [element.text() for element in document.elements]

                # getting the SCDC ID
                scdc = data[data.index("SID:") + 1] # correction scdc is SID not SCDC ID

                # checking if there are "DISCIPLINARY SANCTIONS" in the PDF
                if "DISCIPLINARY SANCTIONS" in data:
                    # creating a smaller list which only has Disciplinary sanctions
                    start_index = data.index("DISCIPLINARY SANCTIONS")

                    if "MOVEMENT" in data:
                        end_index = data.index("MOVEMENT")
                    else:
                        end_index = data.index("EARNED WORK CREDITS")

                    new_list = data[start_index:end_index]

                # adding no record
                else:
                    with open('disciplinary_sanctions.csv', 'a') as discip_file:
                        csv_writer = csv.writer(discip_file)
                        csv_writer.writerow([scdc, "No Record"])
                        continue

                # checking for no disciplanry sanction case
                if 'NO DISCIPLINARIES DURING CURRENT INCARCERATION' in new_list:
                    with open('disciplinary_sanctions.csv', 'a') as discip_file:
                        csv_writer = csv.writer(discip_file)
                        csv_writer.writerow([scdc, "NO DISCIPLINARIES DURING CURRENT INCARCERATION"])
                        continue

                # checking if there are any page breaks in the list
                elif '@VERSION@' in new_list:
                    # if there is a page break just remove elements with page break like "page 2 of 6"
                    start_index = new_list.index('@VERSION@')

                    while True:
                        end_index = start_index + 6
                        if end_index + 1 <= len(new_list):
                            if "SANCTIONS NOT AVAILABLE IN THE AUTOMATED SYSTEM PRIOR TO JANUARY 2009" == new_list[end_index]:
                                end_index = start_index + 10
                        if end_index <= len(new_list):
                            del new_list[start_index:end_index]
                        else:
                            break

                        try:
                            start_index = new_list.index('@VERSION@', start_index)
                        except ValueError:
                            break

                # removing the headers
                new_list = new_list[3:]

                # date pattern
                date_pattern = r'\b\d{2}/\d{2}/\d{4}\b'

                # traversing through the list and opening the file
                with open('disciplinary_sanctions.csv', 'a') as discip_file:
                    csv_writer = csv.writer(discip_file)
                    row = []
                    detail = []

                    if new_list == 'NO DISCIPLINARIES DURING CURRENT INCARCERATION':
                        csv_writer.writerow([scdc, new_list])

                    else:
                        i = 0
                        while (i < len(new_list)):
                            date_found = re.findall(date_pattern, new_list[i])
                            if date_found:
                                if i != 0:
                                    detail_str = ','.join(detail)
                                    row.append(detail_str)
                                    csv_writer.writerow(row)
                                    row = []
                                    detail = []

                                row.append(scdc)
                                row.append(new_list[i])
                                row.append(new_list[i+1])
                                i += 2

                            else:
                                detail.append(new_list[i])
                                i += 1

                        if row != []:
                            detail_str = ','.join(detail)
                            row.append(detail_str)
                            csv_writer.writerow(row)

            except PDFSyntaxError:
                print(f"PDF does not exist: {pdf_url}")
                continue

## Escapes

In [ ]:
import os
import requests
from pdfminer.pdfdocument import PDFSyntaxError
from py_pdf_parser.loaders import load_file
import re
import pandas as pd
import csv
from tqdm import tqdm

headings = ['SID', 'ESCAPE DATE', 'ESCAPE TYPE', 'ESCAPE SEVERITY']
with open('escapes.csv', 'w') as esc:
    csv_writer = csv.writer(esc)
    csv_writer.writerow(headings)

images_csv = pd.read_csv('images.csv')

base_url = "https://public.doc.state.sc.us/scdc-public/inmateDetails.do?id=%"

with open('escapes.csv', 'a') as discip_file:
    csv_writer = csv.writer(discip_file)

    # traversing through all the IDs in the PDF
    for pdf_id in tqdm(images_csv['ID']):
        pdf_id_str = str(pdf_id).zfill(9)  # Pad with leading zeros to make it 9 digits
        pdf_url = base_url + '2' + pdf_id_str
        response = requests.get(pdf_url)

        with open('temp.pdf', 'wb') as pdf:
            pdf.write(response.content)
            try:
                document = load_file('temp.pdf')

                data = [element.text() for element in document.elements]

                # getting the SCDC ID
                scdc = data[data.index("SID:") + 1]

                if "ESCAPES" in data:
                    start_index = data.index("ESCAPES")
                    end_index = data.index("DISCIPLINARY SANCTIONS") - 1
                    new_list = data[start_index:end_index]

                # adding no record
                else:
                    with open('escapes.csv', 'a') as discip_file:
                        csv_writer = csv.writer(discip_file)
                        csv_writer.writerow([scdc, "No Record"])
                        continue

                # checking for no disciplanry sanction case
                if 'NO ESCAPES DURING CURRENT INCARCERATION' in new_list:
                    with open('escapes.csv', 'a') as discip_file:
                        csv_writer = csv.writer(discip_file)
                        csv_writer.writerow([scdc, "NO ESCAPES DURING CURRENT INCARCERATION"])
                        continue

                # checking if there are any page breaks in the list
                elif '@VERSION@' in new_list:
                    # if there is a page break just remove elements with page break like "page 2 of 6"
                    start_index = new_list.index('@VERSION@')

                    while True:
                        end_index = start_index + 10
                        if end_index <= len(new_list):
                            del new_list[start_index:end_index]
                        else:
                            break

                        try:
                            start_index = new_list.index('@VERSION@', start_index)
                        except ValueError:
                            break

                # removing the headers
                new_list = new_list[4:]

                # date pattern
                date_pattern = r'\b\d{2}/\d{2}/\d{4}\b'

                # traversing through the list and opening the file
                with open('escapes.csv', 'a') as discip_file:
                    csv_writer = csv.writer(discip_file)
                    row = []

                    i = 0
                    while (i < len(new_list)):
                        date_found = re.findall(date_pattern, new_list[i])
                        if date_found:
                            if i != 0:
                                csv_writer.writerow(row)
                                row = []

                            row.append(scdc)
                            row.append(new_list[i])

                            if(i+1 >= len(new_list)):
                                i += 1
                                continue
                            else:
                                row.append(new_list[i+1])

                            if(i+2 >= len(new_list)):
                                i += 1
                                continue
                            else:
                                row.append(new_list[i+2])
                            i += 3
                        else:
                            i += 1
                    if row != []:
                        csv_writer.writerow(row)

            except PDFSyntaxError:
                print(f"PDF does not exist: {pdf_url}")
                continue

## Adding image URLs to the detail csv file

In [ ]:
import pandas as pd
import re

image = pd.read_csv('images.csv')

det = pd.read_csv('sc_inmate_details.csv')

image_urls = []
pattern = r"\((\d+)\)"
for name in det['NAME']:
    match = re.search(pattern, name)
    scdc = match.group(1)
    img_url = image.loc[image['ID'] == int(scdc), 'Image URL'].values[0]
    image_urls.append(img_url)

det['Image URL'] = image_urls

det.to_csv('sc_incarcerated_details.csv', index=False)